In [73]:
import pandas as pd
import os
import FinanceDataReader as fdr
import numpy as np
import math
import sklearn
import sklearn.preprocessing
import datetime
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm


In [74]:
path = './'
list_name = 'stock_list.csv'
stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list = stock_list.sort_values("종목코드")
stock_list

,종목명,종목코드,상장시장
109,메리츠화재,000060,KOSPI
126,하이트진로,000080,KOSPI
67,유한양행,000100,KOSPI
69,CJ대한통운,000120,KOSPI
156,두산,000150,KOSPI
...,...,...,...
328,넥스틴,348210,KOSDAQ
31,하이브,352820,KOSPI
199,솔브레인,357780,KOSDAQ
176,티와이홀딩스,363280,KOSPI


In [75]:
data = pd.DataFrame()
volume = pd.DataFrame()
high = pd.DataFrame()
low = pd.DataFrame()

start_date = '20160104'
end_date = '20211106'
for stock_code in stock_list["종목코드"]:
    stock = fdr.DataReader(stock_code, start = start_date, end = end_date)
    data = pd.concat([data,stock["Close"]], axis=1)
    volume = pd.concat([volume, stock["Volume"]], axis=1)
    high = pd.concat([high,stock["High"]], axis=1)
    low = pd.concat([low,stock["Low"]], axis=1)

data.columns = stock_list["종목코드"]
volume.columns = stock_list["종목코드"]
high.columns = stock_list["종목코드"]
low.columns = stock_list["종목코드"]

In [76]:
data_diff = data.iloc[1:, :].values - data.iloc[:-1, :]

In [77]:
from prophet import Prophet

In [78]:
def prophet_pred_model(train):
    if train["y"].isnull().sum() > 0:
        train = train.dropna(axis=0)
    m = Prophet(daily_seasonality=True, yearly_seasonality=True)
    m.fit(train)
    future_prices = m.make_future_dataframe(periods=7)
    pred = m.predict(future_prices[-5:])
#     m.plot_components(pred)
    return pred["yhat"]

In [79]:
result = []
for index in tqdm(range(370)):
    pred_list = None
    train = data_diff.iloc[:, index].reset_index()
    train.columns = ["ds", "y"]
    pred = prophet_pred_model(train)
    result.append(pred)
result

  0%|          | 0/370 [00:00<?, ?it/s]

Initial log joint probability = -11.5925
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2544.72   0.000922892       104.672        6.59       0.659      131   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     106       2544.79   0.000333305        74.558   3.682e-06       0.001      177  LS failed, Hessian reset 
     182       2544.81   2.96139e-05       69.5246   3.071e-07       0.001      307  LS failed, Hessian reset 
     199       2544.81   1.31638e-06       73.0057      0.7532      0.7532      330   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     208       2544.81   2.19477e-07       77.2337      0.5549      0.5549      341   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


  0%|          | 1/370 [00:01<07:10,  1.17s/it]

Initial log joint probability = -42.607
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      98       1664.31   0.000738999       96.7859   5.403e-06       0.001      166  LS failed, Hessian reset 
      99       1664.33   0.000395484       92.8831           1           1      167   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       1665.83   0.000278753       93.3191           1           1      295   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     239       1665.84   1.06983e-07       86.7304      0.2723      0.7342      352   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


  1%|          | 2/370 [00:02<07:10,  1.17s/it]

...

Initial log joint probability = -42.3031
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      90       233.589   9.84254e-05       97.9978   9.025e-07       0.001      141  LS failed, Hessian reset 
      99       233.595    7.9222e-06       94.5327      0.5161      0.5161      151   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     139       233.596   1.02032e-07       99.9393   1.054e-09       0.001      244  LS failed, Hessian reset 
     142       233.596   6.56306e-09       87.5592      0.3921      0.3921      248   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


100%|█████████▉| 369/370 [07:00<00:01,  1.02s/it]

Initial log joint probability = -4.96646
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      96       3237.71   1.35881e-07       85.1331      0.3404           1      124   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


100%|██████████| 370/370 [07:02<00:00,  1.14s/it]


[0     54.985533
 1    131.509236
 2    125.515085
 3    110.732822
 4     84.184707
 Name: yhat, dtype: float64,
 0    75.105228
 1    62.659302
 2   -15.594561
 3   -71.959169
 4   -73.229754
 Name: yhat, dtype: float64,
 0    226.526310
 1    184.815120
 2    162.912008
 3    112.211086
 4    192.645937
 Name: yhat, dtype: float64,
 0    1483.321535
 1    -573.396442
 2    -531.122700
 3    -565.246080
 4    -327.203437
 Name: yhat, dtype: float64,
 0    675.045490
 1    734.556499
 2    534.675149
 3    531.129274
 4    501.185552
 Name: yhat, dtype: float64,
 0    29.960816
 1    26.912021
 2    -1.722916
 3   -20.523228
 4   -33.605518
 Name: yhat, dtype: float64,
 0     42.090696
 1     74.556316
 2     78.364454
 3     -1.905491
 4    140.568288
 Name: yhat, dtype: float64,
 0    219.448286
 1    311.457977
 2    200.466828
 3    109.940276
 4    119.453109
 Name: yhat, dtype: float64,
 0    296.829235
 1    469.197263
 2    142.473262
 3    237.545820
 4    136.010066
 Name: y

In [80]:
sub = pd.read_csv("sample_submission.csv")
result = pd.DataFrame(result).T
for i in range(1,371):
    sub.iloc[:, i] = np.pad(result.iloc[:, i-1] +  data.iloc[-1, i - 1], (0, 5), 'constant', constant_values=0)
sub

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,...,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,29604.985533,33525.105228,60826.526310,145983.321535,134175.045490,17329.960816,50442.090696,88219.448286,107296.829235,...,51210.731289,54048.258164,85809.270117,34156.589196,26929.877252,50185.025414,390970.217003,249857.894721,26159.603503,17408.577945
1,2021-11-02,29681.509236,33512.659302,60784.815120,143926.603558,134234.556499,17326.912021,50474.556316,88311.457977,107469.197263,...,51318.422583,54035.141835,85717.370278,33378.120322,27672.071194,50056.868505,399976.872170,247411.300271,26617.641593,17355.270127
2,2021-11-03,29675.515085,33434.405439,60762.912008,143968.877300,134034.675149,17298.277084,50478.364454,88200.466828,107142.473262,...,51665.312800,53907.217164,85682.804081,33061.302235,27282.930397,50050.834142,396956.956850,247030.701589,26422.505870,17394.717605
3,2021-11-04,29660.732822,33378.040831,60712.211086,143934.753920,134031.129274,17279.476772,50398.094509,88109.940276,107237.545820,...,52203.158579,53860.644745,85504.219582,32840.490535,27927.669044,50442.328818,400160.180270,248767.636666,26410.712391,17374.941520
4,2021-11-05,29634.184707,33376.770246,60792.645937,144172.796563,134001.185552,17266.394482,50540.568288,88119.453109,107136.010066,...,52694.753982,53769.827559,85711.406491,32666.110017,28080.102986,50213.376497,397348.004566,248364.437358,26337.484954,17330.719628
5,2021-11-29,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,2021-11-30,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,2021-12-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,2021-12-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,2021-12-03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [81]:
valid_data = pd.DataFrame()

start_date = '20211108'
end_date = '20211112'
for stock_code in stock_list["종목코드"]:
    stock = fdr.DataReader(stock_code, start = start_date, end = end_date)
    valid_data = pd.concat([valid_data,stock["Close"]], axis=1)
valid_data.columns = stock_list["종목코드"]
valid_data = valid_data[stock_list.sort_values("종목코드")["종목코드"]]
valid_data

from sklearn.metrics import mean_absolute_error
mean_absolute_error(valid_data, sub.iloc[:5, 1:])

3061.782478307521

In [68]:
sub.to_csv("prophet_5.csv", index=0)

In [ ]:
'''
2951.940868644006
2817.600236005654
2801.6621159447204
2788.0299358650095
2763.855672652166
'''
